<a href="https://colab.research.google.com/github/hectormelo/Machine-Learning-Techniques/blob/main/Taller_4/Taller_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/hectormelo/Machine-Learning-Techniques/main/Banner.png" ><br>
# Machine Learning Techniques - MISIS4219

Primer Semestre - 2024

* **Integrante 1:** Jhon Stewar Rayo Mosquera
* **Integrante 2:** Carlos Raúl de la Rosa Peredo

### Modelo de agrupación

El conjunto de datos "Tech Students' Profile Prediction" contiene información sobre las habilidades, preferencias profesionales y antecedentes académicos de estudiantes en el campo de la tecnología. En este proyecto, se busca aplicar técnicas de agrupación para identificar grupos de estudiantes con perfiles similares, lo que podría ser útil para la orientación profesional y la personalización de programas educativos.

El objetivo de este proyecto es desarrollar un modelo de agrupación que clasifique a los estudiantes en función de sus habilidades, intereses y antecedentes, utilizando los datos proporcionados en el conjunto de datos "Tech Students' Profile Prediction". Este modelo deberá ser capaz de identificar patrones y tendencias en los perfiles de los estudiantes, lo que podría ayudar a las instituciones educativas y a los empleadores a adaptar sus estrategias de enseñanza y contratación.

Para poder realizar este proyecto, se le piden los siguientes análisis:

* Realizar la agrupación con el Algoritmo k-Means. Obtener los centroides e interpretarlos.


**Fuente de datos:** [Tech Students' Profile Prediction](https://www.kaggle.com/datasets/scarecrow2020/tech-students-profile-prediction)

### 0. Importar librerías



In [1]:
# Importamos las librerías requeridas para el desarrollo del modelo

# Manejo de datos y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Algoritmo de cluster
from sklearn.cluster import (
    KMeans,
    DBSCAN,
)

# Metricas de evaluación
from sklearn.metrics import (
    silhouette_score,
    silhouette_samples,
    rand_score,
)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, IncrementalPCA

/home/raul/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### 1. Procesamiento y entendimiento de los datos

In [2]:
# Descargamos el data set

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! mkdir -p content
%cd content
! kaggle datasets download scarecrow2020/tech-students-profile-prediction --force
! unzip -o tech-students-profile-prediction.zip
%cd ../

cp: no se puede efectuar `stat' sobre 'kaggle.json': No existe el archivo o el directorio
/home/raul/Escritorio/extra/misis/ml_tech/Machine-Learning-Assignments/Taller4/content


/home/raul/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


100%|█████████████████████████████████████████| 666k/666k [00:01<00:00, 620kB/s]
100%|█████████████████████████████████████████| 666k/666k [00:01<00:00, 620kB/s]
Archive:  tech-students-profile-prediction.zip
  inflating: dataset-tortuga.csv     
  inflating: to_be_scored_tortuga.csv  
/home/raul/Escritorio/extra/misis/ml_tech/Machine-Learning-Assignments/Taller4


In [3]:
df = pd.read_csv('content/dataset-tortuga.csv')
df.head()

,Unnamed: 0,NAME,USER_ID,HOURS_DATASCIENCE,HOURS_BACKEND,HOURS_FRONTEND,NUM_COURSES_BEGINNER_DATASCIENCE,NUM_COURSES_BEGINNER_BACKEND,NUM_COURSES_BEGINNER_FRONTEND,NUM_COURSES_ADVANCED_DATASCIENCE,NUM_COURSES_ADVANCED_BACKEND,NUM_COURSES_ADVANCED_FRONTEND,AVG_SCORE_DATASCIENCE,AVG_SCORE_BACKEND,AVG_SCORE_FRONTEND,PROFILE
0,28,Stormy Muto,58283940,7.0,39.0,29.0,2.0,4.0,0.0,2.0,5.0,0.0,84.0,74.0,NaN,beginner_front_end
1,81,Carlos Ferro,1357218,32.0,0.0,44.0,2.0,0.0,0.0,0.0,5.0,0.0,67.0,45.0,NaN,beginner_front_end
2,89,Robby Constantini,63212105,45.0,0.0,59.0,0.0,5.0,4.0,0.0,4.0,1.0,NaN,54.0,47.0,advanced_front_end
3,138,Paul Mckenny,23239851,36.0,19.0,28.0,0.0,5.0,7.0,0.0,5.0,3.0,NaN,71.0,89.0,beginner_data_science
4,143,Jean Webb,72234478,61.0,78.0,38.0,6.0,11.0,0.0,4.0,3.0,0.0,66.0,85.0,NaN,advanced_front_end


In [6]:
# Existen 13 columnas en el dataset con valores númericos que pueden ser utilizados
# para perfilar a los estudiantes. Dichas columnas cuentan con valores nulos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        20000 non-null  int64  
 1   NAME                              20000 non-null  object 
 2   USER_ID                           20000 non-null  int64  
 3   HOURS_DATASCIENCE                 19986 non-null  float64
 4   HOURS_BACKEND                     19947 non-null  float64
 5   HOURS_FRONTEND                    19984 non-null  float64
 6   NUM_COURSES_BEGINNER_DATASCIENCE  19974 non-null  float64
 7   NUM_COURSES_BEGINNER_BACKEND      19982 non-null  float64
 8   NUM_COURSES_BEGINNER_FRONTEND     19961 non-null  float64
 9   NUM_COURSES_ADVANCED_DATASCIENCE  19998 non-null  float64
 10  NUM_COURSES_ADVANCED_BACKEND      19992 non-null  float64
 11  NUM_COURSES_ADVANCED_FRONTEND     19963 non-null  float64
 12  AVG_

In [7]:
# Se asumirá que los valores nulos en las columnas se debe a que
# el estudiante no ha completado la información porque no cuenta 
# 'experencia' en esos rubros. 
# Se procederá a rellenar los valores nulos con 0.

df = df.fillna(0)